---
# Beleg Prescriptive Analytics with Python <br>- Jannis Kaliske (4886335) -
Traveling Salesman Problem with precedence constraints with<br>
TabuSearch - MetaHeuristic
---

---

Das vorgestellte logistische Problem ist ein sogenanntes "Traveling Salesman Problem with precedence constrains". Diese Herausforderung der Planung basiert darauf, dass in einem Pool eine gewisse Anzahl an Anlaufpunkte gegeben sind. Es muss jeder einzelne Punkt genau ein Mal angelaufen werden. Am Ende soll man zum Anfangspunkt zurückgekommen sein. Die Inputdaten beinhalten einerseits Dateiname, die Anzahl der anzulaufenden Punkte und die Wegematrix. Die Anzahl der Punkte unterscheidet sich von der Anzahl der Spalten/Zeilen der Wegematrix, da zusätzlich Start und Endpunkt in der Matrix enthalten sind. Demzufolge muss die Tour immer in Knoten 0 (erste Zeile) beginnen und endet immer in Knoten 20 (letzte Spalte). Knoten 20 ist der selbe Punkt wie Knoten 0, wurde jedoch hinzugefügt, um es mathematisch einfacher zu beschreiben. Zuletzt ist darauf hinzuweisen, dass gewissen Verbindungen eine Bewertung von "-1" aufweisen, was einen verbotenen Weg darstellt -> gewisse Ristriktionen im Traveling Salesman Problem: precedence constraints.

Eine kurze Übersicht der benötigten Dateien/Klassen:<br>
-   SolutionFile.ipynb      --> gibt übersichtlich den Programmierverlauf an, den Start des Algorithmus bitte über die letzte Codezelle in diesem Dokument<br>
-   InputData.py            --> mit Klasse InputData wird Wegematrix generiert<br>
-   EvaluationLogsic.py     --> mit Klasse EvaluationLogic wird Evaluierung einer Permutation durchgeführt<br>
-   BestStartSolution.py    --> mit Klasse bestStartSolution wird eine Startlösung generiert<br>
-   Solver.py               --> mit Klasse Solver und den Funktionen generateNeighborhood, tabuSearch wird MetaHeuristik gestartet und gesteuert<br>
-   OutputData.py           --> mit Klasse outputAlgorithm wird eine csv Datei entsprechend der Vorgaben generiert und in einem Unterordner ("Solutions") entsprechend der Namensvorgabe abgespeichert<br>
-   Ordner TestInstanzen
-   Ordner Solutions

Im Folgenden werden die Wegematrizen in den Algorithmus geladen. Hierbei wird auf die DataFrame Funktion von dem Package Pandas zurückgegriffen, weil dieses eine relativ einfach zu handhabende Struktur aufweist. Es wurde ein eigenes .py Dokument erstellt, um eine übersichtliche Dateistruktur aufzubauen. Dieser InputData Klasse wird der Dateiname (path) als Variale übergeben.
Die Testinstanzen sind in einem Unterordner gespeichert, deshalb muss dieser zuerst angewählt werden mit #[1]. Anschließend wird die Datei mit dem Package json geladen. Auf die anderen Variable abseits der Wegematrix wurde verzichtet, da die Länge/Anlaufpunktzahl auch einfach über die Länge der Matrix einer beliebigen Reihe errechnet werden kann.
<br><br>
Für die Nutzung des Algorithmus mit den Testdaten bitte diese Dateien unbedingt in de Unterordner "Testinstanzen" kopieren. 
<br><br>
In #[2] wird die Achsenbeschriftung generiert, damit das Handhaben mit dem DataFrame später einfacher wird<br>
In #[3] wird die in self.data gespeicherte matrix in ein Dataframe geladen und die Achsenbeschriftung übergeben.

In [ ]:
##InputData.py
import json
import pandas as pd

class InputData:

    def __init__(self, path):
        #Eingabe des Verzeichnisses der Touren-Daten
        self.path = path
        self.DataLoad()

    def DataLoad(self):
        var = ['Testinstanzen']
        var.append(self.path)

        #[1]    #Eingabe des Verzeichnisses der Touren-Daten
        path = '\\'.join(var)

        #Einlesen der Daten aus der Touren-Daten-Datei und abspeichern in Variable "data"
        with open(path) as f:
            inputData = json.load(f)

        #Abspeichern der Daten
        self.data = inputData['Distances']

        #[2] #Generierung eines Arrays mit den Zahlen 0-(Anzahl Punkte im Tourenplan) zur Achsenbeschriftung, um eindeutiger und einfacher Matrix nachvollziehen zu können
        axis = [i for i in range(len(self.data[0]))]

        #[3] #Erstellung einer Matrix mittels Pandas-DataFrame, da mit diesem Kontrukt relativ leicht gearbeitet werden kann
        self.matrix = pd.DataFrame()
        for row in self.data:
            self.matrix = pd.DataFrame(self.data, columns = axis, index = axis) #Einfügen der Daten mit Beschriftung der Achsen x,y 
\\ #diese zwei Zeichen bitte nicht beachten

---

Nachdem der Input generiert wurde, muss der Evaluierungsalgorithmus gecoded werden, der mit der Kostenfunktion die Güte einer variablen Permutation bestimmt.
Dazu wurde die Datei EvaluationLogic.py geschrieben. Für die Algorithmus ist die Permutation (order) und die Wegematrix (matrix) wichtig, die ihm als Variablen übergeben werden. Bitte folgende Notizen im Code lesen:

In [ ]:
##EvaluationLogic.py
import math

class EvaluationLogic:

    def __init__(self):
        self.path_sum = 0

    def SolutionFinder(self, order, matrix):    
    #Einführen von Lauf-/Zählvariablen
        self.path_sum = 0   #Gibt die Summe der Wegkosten an
        i = 0               #Iteriert wird über jedes Element des wieder zusammengesetzten Arrays order
        zaehlvariable = 1   #Wird benötigt, um Abbruchkriterium einzubauen, da sonst ein Fehler geworfen wird, wenn Iteration beim letzten Element ist, da in for-Schleife die Wegkosten
                            #vom aktuellen Element zum nächsten Element im Array aufsummiert werden. Beim letzten Element im Array gibt es allerdings kein darauffolgendes Element.
                            #deshalb wird ein Abbruchkriterium 'if zaehlvariable == len(data[0])' eingebaut

        for i in order: #über jedes Element im Array order iteriert
            if zaehlvariable == len(matrix[0]): #Abbruchkriterium
                break #wenn dies eintritt, wird for-Schleife abgebrochen und der Code außerhalb fortgesetzt
            elif matrix.iloc[i, order[zaehlvariable]] == -1: #Zweites Abbruchkriterium: wenn eine Verbindung eine Bewertun von '-1' hat, bedeutet dies, dass diese Verbindung unzulässig ist.
                self.path_sum = math.inf #wenn Verbindung unzulässig, wird den Wegkosten dieser Permutation der WErt Unendlich zugewiesen und for-Schleife abgebrochen, nächste Permutation wird gestartet
                break
            else: #wenn kein Abbruchkriterium erfüllt, wird folgender Code ausgeführt
                self.path_sum = self.path_sum + matrix.iloc[i, order[zaehlvariable]]  #Wegkosten werden kumuliert, indem zu den bisherigen Wegkosten die Kantenbewertung vom Punkt i zum darauffolgenden Punkt im Array addiert werden
                                                                        #hierbei muss allerdings der darauffolgende Punkt mit 'order[zaehlvariable]' angesprochen werden, da 'i+1' lediglich zur inkrementellen Erhöhung 
                                                                        #des Wertes i führt und nicht zur nächsten Stelle im Array. Dafür wurde die Zählvariable 'zaehlvariable' eingeführt
                zaehlvariable += 1 #Zählvariable inkrementell erhöhen, für nächsten Iterationsschritt
        return self.path_sum 
\\ #diese zwei Zeichen bitte nicht beachten

---

Nachdem Input und Evaluierung einer Lösung implementiert wurden, kann nun eine erste Startlösung generiert werden. Ich habe mich hierbei für die Auswahl der besten Permutation aus drei verscheidenen Generierungen von Startlösungen entschieden: FirstComeForstServe (FCFS), Zufallsgenerierung, ShortestNextWayTime (aka ShortestProcessingTime). Die Generierung der FCFS Permutation ist sehr simpel und wird als erstes vorgenommen. Hierbei werden lediglich die Zahlen von 0 bis zum Endpunkt aufsteigend aufgelistet. Dies entspricht der Regel FCFS, die besagt, dass der Erste zuerst angenmmen wird und anschließend der zweite, anhand der Nummerierung wird dies deutlich. <br>
Anschließend wurde die Zufallsgenerierung implementiert. Diese ist folgenden ausschnittsweise dargestellt: es werden zufällig 200 Zahlen generiert, die anschließend als Seeds benutzt werden, um 200 Permutationen zu erstellen, die anschließend auf ihre Güte mittles EvaluationLogic geprüft werden. Die beste Permutation wird an den startSolutionPool weitergegeben, der alle besten Lösungen der drei StartAlgorithmen aufnimmt.

In [ ]:
        #generate random solutions, 200 zufällige Zahlen mit seed jedoch reproduzierbar
        random.seed(self.seed)
        random_numbers = [random.randint(0, 10000) for x in range(200)]
        randomZwischenspeicher = []
        for i in random_numbers:#nutze alle Zahlen als unterschiedliche Seeds
            #Generierung eines Arrays mit den Zahlen 0-(Anzahl Punkte im Tourenplan) zur Achsenbeschriftung, um eindeutiger und einfacher Matrix nachvollziehen zu können
            axis = [i for i in range(len(self.matrix[0]))]

            #Aus Achsenbeschriftung werden Start- und Endpunkt gelöscht, da diese immer an der selben Stelle sein müssen und beim Zufallsgenerieren sonst nicht End- und Anfangspunkt wären
            axis.remove(0)
            axis.remove(len(self.matrix[0]) - 1)
            size=len(self.matrix)

            #Randomisiert wird der Mittelteil des Arrays gemischt, um eine zufällige Lösung zu erzeugen
            first_order = random.sample(axis, len(self.matrix[0])-2)

            #Array wird wieder zusammengesetzt, indem Start- und Endpunkt hinzugefügt werden. Dadurch kann sichergestellt werden, dass Punkt 0 immer Anfang und Ende darstellt
            order = [0]
            for k in first_order:
                order.append(k)
            order.append(size-1)

            if len(randomZwischenspeicher) == 0:
                randomZwischenspeicher.append(order)
            else:
                if EvaluationLogic().SolutionFinder(order, self.matrix) < EvaluationLogic().SolutionFinder(randomZwischenspeicher[-1], self.matrix):
                    randomZwischenspeicher.append(order)

        self.startSolutionPool.append(deepcopy(randomZwischenspeicher[-1])) 
\\ #diese zwei Zeichen bitte nicht beachten

Zuletzt wird über ShortestProcessingTime eine Lösung generiert: für jeden Punkt wird der Nachfolger ausgewählt, der den kürzesten Weg hat. Dafür wurden zuerst alle Wegbewertungen von "-1" auf math.inf gesetzt, um einen unendlich hohen Wert zu erzeugen. Dies ist nötig, da bei der Suche nach dem kürzesten Weg eine Wegbewertung von -1 selbstverständlich als die beste Lösung erkannt wird (#[1]). Zudem wird dies auch mit der Hauptdiagonalen gemacht, die diese Bewertungen null sind und zu cycling führen würden. <br>
Anschließend wird beginnend in der ersten Zeile nach dem Nachfolger gesucht, der die geringste Wegbewertung hat. Dieser wird anschließend über die index-Funktion angepeilt und als neue Start-Zeile gewählt. Zuvor wird allerdings diese Spalte gelöscht, da dieser Punkt nun angelaufen wurde. Nach dieser Regel wird bis zum Ende durchgelaufen und die finale Permutation mit Start und Endpunkten wieder versehen (#[2]). Dies ist nötig, da diese zuvor entfernt wurden, damit der Algorithmus den letzten Punkt (20) bereits vorher wahrnimmt. Da dies der Zielpunkt sein muss, wird hier vorgesorgt.

In [ ]:
        #generate ShortestProcessingTime
        matrice = deepcopy(self.matrix)

        for i in range(len(matrice[0])-1):
            matrice[i][i] = math.inf #soll anlaufen des selben Punktes verhindern
        matrice.replace(-1, math.inf, inplace = True) #setze unmögliche Weglängen auf unenedlich

        reihenfolge= []
        zeile = 0
        i = 0

        #letze Spalte löschen, um vorzeitiges Ende zu verhindern (letzter Anlaufpunkt ist immer die letzte Spalte)
        matrice.drop(matrice.tail(1).index,inplace=True)
        matrice = matrice.iloc[: , :-1] 
        lenght = len(matrice[0]-1)

        while i < (lenght-1): #suche für jede Zeile nach dem besten Nachfolger mit der geringsten Wegbewertung
            matrice.iloc[zeile].idxmin()
            zeile_alt=deepcopy(zeile)
            reihenfolge.append(matrice.loc[zeile].idxmin())
        
            zeile = matrice.loc[zeile].idxmin()
            del matrice[zeile_alt] #lösche angelaufenen Punkt aus Matrix (Spalte)
            i+=1

        reihenfolge.append(len(matrice))
        reihenfolge.insert(0, 0)

        self.startSolutionPool.append(deepcopy(reihenfolge)) 
\\ #diese zwei Zeichen bitte nicht beachten

Am Ende werden lediglich die drei Werte aus dem startSolutionPool verglichen und der geringste Wert (durch EvaluationLogic ausgewertet) als beste StartPermutation zurückgegeben (siehe BestStartSolution.py).

---

Anschließend startet der eigentliche Part: nach der Problem-Implementierung und der InitialLösungsGenerierung wird nun die InitialLösung an die MetaHeuristik weitergegeben. Neben der InitialLösung, dem Path (Name der Testinstanz) wird der MetaHeuristik zwei Parameter und die Wegematrix mitgegeben. Damit kann der TabuSearch Algorithmus gestartet werden. Die veränderbaren Parameter sind 1) maxTabuListLength und 2) maxIterations. 1) wirk hierbei als Intensivierungs- und Diversifikations Instrument und steuert, welche Tausche nicht erlaubt werden (mehr dazu im Folgenden). 2) Gibt an, wieviele Iterationen pro Nachbarschaft pro TabuSearch Iteration durchgeführt werden sollen. Beides hat Einfluss darauf, wie schnell und gut Lösungen gefunden werden.

Zuerst jedoch einige Hinweise und Erklärungen zur verwendeten MetaHeuristik - TabuSearch. Die Idee von TabuSearch ist relativ simpel. Es wird eine Liste instanziiert, die bereits getätigte Tausche verbietet und so eine Rückkehr zu einem lokalen Optimum verhindern soll. Solange ein gewisses Abbruchkriterium nicht erreicht ist, werden mögliche Lösungen aus der Nachbarschaftssuche gesammelt und geprüft, ob sie tabu sind (in TabuListe enthalten) oder ob ein gewisses Aspirationskriterium erfüllt ist, die die TabuListe umgeht. <br><br>
Im Folgenden werden zwei Codebeispiele gezeigt, die eine mögliche Implementierung der Metaheuristik zeigen: die erste Version ist eine etwas leistungsfähigere, jedoch nicht so akkurate Implementierung von TabuSearch. Genauer gesagt werden die TabuListe die generierten Permutationen aufgenommen. Wenn der Algorithmus eine Permutation als die beste der Nachbarschaft einstuft, fügt er sie der TabuListe hinzu und in der nächsten Iteration darf diese Permutation nicht erreicht werden. Bitte beachten Sie die Kommentare besonders am Anfang:

In [ ]:
#Diese Version von TabuSearch entspricht NICHT vollständig der Idee, da die Permutationen als tabu abgespeichert werden.
#Diese Version soll nur der Anschauung dienen und liegt dem Beleg zwar bei, soll jedoch NICHT als finale Abgabe verstanden werden.
#Diese Version liegt dem Beleg bei und nutzt bei den File-Namen immer "Part...", damit keine Verwechslungen entstehen
def tabuSearch(self, firstSolution, matrix, path, maxTabuListLength=5, seedf = 50):
        self.firstSolution = firstSolution
        self.matrix = matrix
        self.path = path
        tabuList2 = [deepcopy(firstSolution)]
        Solution = deepcopy(firstSolution)
        self.solutPoool.append(deepcopy(firstSolution))
        self.maxTabuListLength = maxTabuListLength
        self.seedf = seedf
        time_needed=default_timer()
        timer = 0
        iteration = 1

        if len(self.matrix[0])<50:
            maxTabuSearchTime = 28
        else:
            maxTabuSearchTime = 290
            
        while timer < maxTabuSearchTime:
            print(f"\nTabu-Search, Iteration {iteration}")
            iteration+=1

            while len(tabuList2) > self.maxTabuListLength:
                tabuList2.pop(0)

            di = self.generateNeighboorhood(deepcopy(Solution), self.matrix, tabuList2, type='insertion', maxIterations=1000, seed=self.seedf)
            ds = self.generateNeighboorhood(deepcopy(Solution), self.matrix, tabuList2, type='swap', maxIterations=1000, seed=self.seedf)

            if EvaluationLogic().SolutionFinder(di, self.matrix) < EvaluationLogic().SolutionFinder(ds, self.matrix):   
                Solution=deepcopy(di)
                tabuList2.append(Solution)
                self.solutPoool.append(deepcopy(Solution))

            else:
                Solution=deepcopy(ds)                
                tabuList2.append(Solution)
                self.solutPoool.append(deepcopy(Solution))

            self.solutPoool.append(deepcopy(Solution))
            timer = default_timer() - time_needed 
\\ #diese zwei Zeichen bitte nicht beachten

Diese Version entspricht nicht ganz der Idee der MetaHeuristik, denn diese besgt, dass die Tausche selbst und nicht die Permutationen auf die TabuListe kommen sollen. Wenn Stelle 3 und 7 getauscht werden, sollen diese in den nächsten zB fünf Iterationen nicht wieder zurückgetauscht werden. Dies könnte zu sog. Cycling führen und immer das selbe lokale Optimum herbeiführen. Deshalb wurde noch eine zweite Variante implementiert, in der immer die Stellen der Tausche zurückgegeben werden und damit verboten werden. Dies macht auch die Implementierung des sog. Aspirationskriteriums sinnvoll, das laut der Litaratur meist besagt, dass ein Tausch trotz Verbot durch die TabuListe angenommen werden soll, wenn dadurch ein besserer Funktionswert entsteht [1]. Wenn man die Permutationen auf die TabuListe setzen würde, ist es unmöglich, einen besseren Funktionswert als bisher zu erreichen, wenn alle auf der TabuListe enthaltenen Werte (Permutationen) jedoch schon besucht wurden, da sie mal die besten Permutationen ihrer Nachbarschaft waren und bereits in den LösungsPool aufgenommen wurden.

Demzufolge wird nun die folgende TabuSearch Implmentierung erklärt. Im ersten Absatz werden lediglich lokale Laufvariablen, die TabuListe, gewisse Zwischenspeicher und der Lösungspool (self.solutPoool) angelegt. Anschließend werden die vereinbarten Zeiten für die Rechenzeit für die verschiedenen Probleme durch die Länge der Matrx bestimmt. Anschließend beginnt der eignetliche Algorithmus, solange (while-Bedingung) die gemessene Zeit unter der vereibarten Zeit liegt, soll TabuSearch iteriert werden. Es werden beide implementierten Nachbarschaften durchsucht (mehr dazu später) und anschließend die besten Ergebnisse dieser verglichen. Wenn die swap-Nachbarschaft zum besseren Ergebnis geführt hat, wird diese zum SolutionPool hinzugefügt und wird die neue StartPermutation für die nächste Tabu-Iteration und wenn nicht, dann die insertion-Lösung. Der bessere Tausch wird der TabuListe hinzufügt und darf in der nächsten Zeit nicht wiederholt werden (gesteuert durch User eingegebenen Parameter).

In [ ]:
#Richtige TabuSearch Version, die auch implementiert wurde und in der letzten Zelle dieses Dokuments gestartet werden kann
def tabuSearch(self, firstSolution, matrix, path, maxTabuListLength=5, maxIterations = 500):
        self.firstSolution = firstSolution
        self.matrix = matrix
        self.path = path
        tabuList = [] #Kurzzeitgedächtnis, nimmt entsprechend des Parameters eine gewisse Anzahl von durchgeführten Tauschen auf, die danach verboetn werden, bis sie rausgelöscht werden
        Solution = deepcopy(firstSolution) #Startwert der ersten Iteration
        self.solutPoool.append(deepcopy(firstSolution)) #Startlösung dem Langzeitgedächtnis aka LösungsPool hinzufügen
        self.maxTabuListLength = maxTabuListLength
        self.maxIterations = maxIterations
        time_needed=default_timer() #starte Zeitabgleich/Timer
        timer = 0
        iteration = 1
        tauschKombination = []
        xtauschKombination = []

        if len(self.matrix[0])<50: #Vorgabe der Rechenzeit abzüglich Pauschal Erfahrungswert an Sekunden zur Beendigung der letzten Iteration und Durchsuchung des Langzeitgedächtnisses nach der besten Lösung
            maxTabuSearchTime = 28
        else:
            maxTabuSearchTime = 280
        
        while timer < maxTabuSearchTime: #Abbruchkriterium in Form eines Zeittimers
            print(f"\nTabu-Search, Iteration {iteration}")
            iteration+=1
            
            #durchsuche beide Nachbarschaften nach der besten Lösung, die nicht auf tabuList ist ODER aspirationskriterium erfüllt
            di = self.generateNeighboorhood(deepcopy(Solution), self.matrix, tabuList, type='swap', maxIterations=self.maxIterations)
            ds = self.generateNeighboorhood(deepcopy(Solution), self.matrix, tabuList, type='insertion', maxIterations=self.maxIterations)
            
            #Auswertung der Ergebnisse
            if EvaluationLogic().SolutionFinder(di, self.matrix) <= EvaluationLogic().SolutionFinder(ds[0], self.matrix):#falls swap Ergebnis besser oder gleich insertion Ergebnis ist:   
                tauschKombination = []

                for index, (first, second) in enumerate(zip(Solution, di)): #herausfinden, welche Tausche gemacht wurden, dazu wird die letzte Lösung mit der neuen Lösung verglichen und geänderte Stellen in tauschKombination gespeichert
                    if first != second:
                        tauschKombination.append(index)

                xtauschKombination = tauschKombination[0], tauschKombination[1] #Liste wird in Tupel konvertiert, um eindeutig zuordnen zu können
                tabuList.append(xtauschKombination) #Kurzzeitgedächtnis erweitern mit letztem Tupel
                Solution=deepcopy(di) #Startwert der kommenden Iteration
                self.solutPoool.append(deepcopy(Solution)) #dem Langzeitgedächtnis die lokale beste Lösung hinzufügen

            else: #insertion Wert war besser
                tauschKombination = deepcopy(ds[1]) #beste Tauschkombination ist an der zweiten Stelle der Rückgabeliste gepseichert (Besonderheit im Vergleich zu swap)
                tabuList.append(tauschKombination) #Kurzzeitgedächtnis erweitern mit letztem Tupel
                
                Solution=deepcopy(ds[0]) #Startwert der kommenden Iteration
                self.solutPoool.append(deepcopy(Solution))  #dem Langzeitgedächtnis die lokale beste Lösung hinzufügen

            timer = default_timer() - time_needed #Timer aktualisieren


            while len(tabuList) > self.maxTabuListLength: #wenn TabuListe länger als Vorgabewert, lösche den ältesten Eintrag an erster Stelle
                tabuList.pop(0) 
\\ #diese zwei Zeichen bitte nicht beachten

Der Timer wird anschließend upgedated  und falls die maximale Länge der TabuListe erreicht wurde, wird das älteste Ergebnis aus der Liste gelöscht. Dieses Vorgehen wiederholt sich so lange bis der Timer über der Zeit ist. Ist dies der Fall, wird der komplette Lösungsraum (solutionPool) nach der besten Lösung abgesucht und zurückgegeben. Dazu wird die lokaleVariable Zwischenspeicher generiert, in die alle Wegzeiten hinzugefügt werden. Anschließend wird mit einer einfachen Funktion der Index des geringsten Werts aus der gesamten Liste gesucht. Dieser ist sogleich auch die Stelle von der zugehörigen Permutation im SolutionPool. Damit kann schnell die beste Permutation der gesamten Heuristik bestimmt und zurückgegeben werden:

In [ ]:
        #Abbruchkriterium wurde erreicht, Zeit ist abgelaufen: durchsuche gesamten LangzeitGedächtnis - Lösungsraum
        zwischenSpeicher = []
        for i in self.solutPoool:
            zwischenSpeicher.append(EvaluationLogic().SolutionFinder(i, self.matrix)) #errechne für alle in SolutionPool enthaltenen Lösungen die Weglänge

        index_min = min(range(len(zwischenSpeicher)), key=zwischenSpeicher.__getitem__) #Suche nach dem Index mit dem geringsten Lösungswert
        bestFinalSolution = deepcopy(self.solutPoool[index_min]) #die gesamte-beste Lösung

        print(f'\nDie beste globale Lösung der MetaHeuristik ist {bestFinalSolution} mit {EvaluationLogic().SolutionFinder(deepcopy(bestFinalSolution), self.matrix)}.')
        print(f'Dafür hat die MetaHeuristik {timer} Sekunden an Rechenzeit benötigt.')

        outputAlgorithm(bestFinalSolution, self.matrix, self.path).startOutput() #starte Output auf Basis der gesamt-besten Lösung
        
        return bestFinalSolution 
\\ #diese zwei Zeichen bitte nicht beachten

Ein wesentlicher Bestandteil der Heuristik ist die Suche in den Nachbarschaften. DIese wird im Folgenden erklärt. Entschieden wurde sich für zwei Nachbarschaften: swap und insertion. Beide werden in der selben Solver-Klasse der Datei Solver.py unter der Methode .generateNeighborhood() aufgerufen. 

Im Wesentlichen läuft bei der swap-Nachbarschaft alles darauf hinaus, dass jedes Element mit jedem anderen Element getauscht und ausgewertet wird. Da es sich hier um eine Symmetrie handelt, werden nur die Tausche durchgeführt für die i < j gilt. Dadurch kann sehr viel Rechenleistung gespart werden (der Tausch von Stelle 3 und 7 ergibt das Selbe wie der Tausch von 7 mit 3).<br>
Es werden Indices festgelegt, welche Stellen getauscht werden. Anschließend wird getestet, ob dieser Tausch tabu ist oder nicht (Abgleich mit TabuListe) ODER ob das zuvor erwähnte Apsirationskriterium erreicht wird. Letzteres wird relativ simpel durch iterative Verbesserungen ermittelt: Variable aspirationskriterium hat den Anfangswert math.inf (unendlich) und nur verbessernde Werte werden aufgenommen und dürfen die TabuListe umgehen. Dieser Wert wird nach jeder Verbesserung überschrieben mit Funktionswert und Permutation. Dies wird solange durchgeführt, bis entweder die gesamte Nachbarschaft durchsucht wurde oder die Anzahl maxIterations, die der Funktion beim Aufruf mitgegeben wurde, erreicht wurde. Anschließend wird ähnlich wie zuvpr mittels "zwischenspeicher" die beste lokale Permutation gesucht, die entweder nicht tabu oder das aspirationskriterium erfüllt.

In [ ]:
class Solver:

    def __init__(self, path):
        self.path = path  
        self.solutPoool = [] #Langzeitgedächtnis, nimmt langfristig alle lokal besten Lösungen auf
        

    def generateNeighboorhood(self, firstSolution, matrix, tabuList, type = 'swap', maxIterations=10000):
        self.matrix = matrix
        self.firstSolution = firstSolution
        self.type = type
        self.maxIterations = maxIterations
        self.tabuList = tabuList
        localBestSolution = []
        it=0
        #Aspirationskriterium initialisiert mit einem maximalen Wert und bei einer Verbesserung wird die List überschrieben
        aspirationskriterium = [EvaluationLogic().SolutionFinder(deepcopy(firstSolution), self.matrix), deepcopy(firstSolution)]
        

        if type == "swap": #swap Nachbarschaft wird gestartet
            localBestSolution.clear()

            for i in range(1, len(self.firstSolution)-1):
                for j in range(1, len(self.firstSolution)-1):
                    if it == self.maxIterations:
                        break
                    else:
                        it+=1
                        if i < j:
                            indexA = j
                            indexB = i
                            nextSolution = list(self.firstSolution) # create a copy of the permutation

                            #swap der Punkte an den indice STellen wird vorgenommen
                            nextSolution[indexA] = deepcopy(self.firstSolution[indexB])
                            nextSolution[indexB] = deepcopy(self.firstSolution[indexA])

                            tauschKombi = (i, j)
                            
                            #Abgleich mit TabuListe ODER Aspirationskriterium ist wahr, falls aktuelle Lösung besser als alle bisher, wird aspirationskriterium upgedated
                            if (tauschKombi not in self.tabuList) or (EvaluationLogic().SolutionFinder(deepcopy(nextSolution), self.matrix) < aspirationskriterium[0])==True:
                                    localBestSolution.append(nextSolution)
                                    if aspirationskriterium[0] > EvaluationLogic().SolutionFinder(nextSolution, self.matrix):
                                        aspirationskriterium[0] = EvaluationLogic().SolutionFinder(nextSolution, self.matrix)
                                        aspirationskriterium[1] = nextSolution
            
            #durchsuchen der localBestSolution Liste nach der besten lokalen Lösung der Nachbarschaft
            zwischenSpeicher = []
            for i in localBestSolution:
                zwischenSpeicher.append(EvaluationLogic().SolutionFinder(deepcopy(i), self.matrix))

            index_min = min(range(len(zwischenSpeicher)), key=zwischenSpeicher.__getitem__) #Suche nach dem Index mit dem geringsten Lösungswert
            singleLocalBestSolution = deepcopy(localBestSolution[index_min])

            print(f'eine Weglänge von {EvaluationLogic().SolutionFinder(deepcopy(singleLocalBestSolution), self.matrix)} über {singleLocalBestSolution} wurde mit swap erreicht')
            return singleLocalBestSolution 
\\ #diese zwei Zeichen bitte nicht beachten

Zurückgegeben wird die beste Permutation der Nachbarschaft. Bisher wurde Folgendes noch nicht erwähnt: das Update der TabuListe erfolgt in der Funktion tabuSearch, nachdem die beste Nachbarschafts-Permutation erzeugt wurde. Für den Swap-Algorithmus ist dies sehr einfach, da durch eine kleine und einfache Logik jede Stelle der neuen Permutation wird der Stelle der vorherigen Permutation verglichen wird. Die Indices ergeben den durchgeführten Tausch:

In [ ]:
        tauschKombination = []

        for index, (first, second) in enumerate(zip(Solution, di)): #herausfinden, welche Tausche gemacht wurden, dazu wird die letzte Lösung mit der neuen Lösung verglichen und geänderte Stellen in tauschKombination gespeichert
            if first != second:
                tauschKombination.append(index)

        xtauschKombination = tauschKombination[0], tauschKombination[1] #Liste wird in Tupel konvertiert, um eindeutig zuordnen zu können 
\\ #diese zwei Zeichen bitte nicht beachten

Dies kann bei der Insertion jedoch nicht angewandt werden, die sich hier oftmals mehr als nur eine Stelle ändern. Insertion an sich wählt eine Zahl und eine Stelle aus. Diese Zahl wird anschließend an diese Stelle gerückt und der Rest aufgefüllt. Im Folgenden ist der Algorithmus dargestellt:

In [ ]:
elif type == 'insertion': #insertion Nachbarschaft wird gestartet
            nextPermut = []
            nextPermuts = []
            localBestSolution.clear()
            firstSolutio = deepcopy(self.firstSolution)

            for i in range(1, len(firstSolutio)-1):
                for j in range(1, len(firstSolutio)-1):
                    if it == self.maxIterations:
                        break
                    if i == j or i == j + 1:
                        continue
                    elif i<j:
                        it+=1
                        nextPermuts.clear()
                        nextPermut.clear()
                        indexA = deepcopy(i)
                        indexB = deepcopy(j)

                        for k in range(len(firstSolutio)):
                            if k == i:
                                continue
                            nextPermut.append(deepcopy(firstSolutio[k]))
                        nextPermut.insert(indexB, deepcopy(firstSolutio[i]))
                        tauschKombi = (i, j) #TauschKombination, die am Ende an die TabuListe weitergegeben wird
                        nextPermuts = [nextPermut, tauschKombi] #Liste besteht aus nächster Permutation und TauschKombination, da die tauschKombi an TabuSearch zurückgegeben werden muss, um TabuListe zu ergänzen

                        #Abgleich mit TabuListe ODER Aspirationskriterium ist wahr, falls aktuelle Lösung besser als alle bisher, wird aspirationskriterium upgedated
                        if (tauschKombi not in self.tabuList) or (EvaluationLogic().SolutionFinder(nextPermuts[0], self.matrix) < aspirationskriterium[0])==True:
                            localBestSolution.append(deepcopy(nextPermuts))
                            nextSolu = EvaluationLogic().SolutionFinder(nextPermuts[0], self.matrix)
                            if aspirationskriterium[0] > nextSolu:
                                aspirationskriterium[0] = nextSolu
                                aspirationskriterium[1] = nextPermuts[0] 
\\ #diese zwei Zeichen bitte nicht beachten

Für jedes Element an jeder Stelle wird die insertion durchgeführt, solange es nicht die maxIterations übersteigt. Am Ende sieht man die selbe Logik wie zuvor: Lösung wird dem lokalen SolutionPool hinzugefügt, sofern er nicht tabu ODER das Aspirationskriterium angenommen wird. Um nun jedoch die Stellen des besten Tauschs aus dem Pool herauszubekommen wird laufend verglichen, ob es einen besseren Wert als unendlich gibt und sobald dieser angenommen wird, wird dieser als neues apsiratoinskriterium auf der Listenstelle [0] übernommen. Jedoch ist die Besonderheit hier, dass gleichzeitig auch die Stellen der Indices/Tauschstellen übernommen werden und gemeinsam als Rückgabewert in der TabuSearchAlgorithmus zurückgegeben werden, um diese der TabuListe hinzuzufügen. Dabei entspricht Rückgabewert ds[1] die TauschKombination, die an die TabuListe weitergegeben wird und ds[0] der besten Permutation aus der Nachbarschaft insertion. Am Ende wird die Lösung dem solutPoool hinzugefügt, sofern dieser besser als die der swap Nachbarschaft ist. Dies geschieht hier:

In [ ]:
                tauschKombination = deepcopy(ds[1]) #beste Tauschkombination ist an der zweiten Stelle der Rückgabeliste gepseichert (Besonderheit im Vergleich zu swap)
                tabuList.append(tauschKombination) #Kurzzeitgedächtnis erweitern mit letztem Tupel
                
                Solution=deepcopy(ds[0]) #Startwert der kommenden Iteration
                self.solutPoool.append(deepcopy(Solution))  #dem Langzeitgedächtnis die lokale beste Lösung hinzufügen
\\ #diese zwei Zeichen bitte nicht beachten

Eine wichtige Erkenntnis ist, dass die Nachbarschaftssuche nicht immer verbessernde Werte zurückliefern muss, sondern aufgrund der TabuListe auch manchmal gar nicht kann. Es wird zwar in der Intensivierungsphase immer der beste Wert weitergeben, kommt es jedoch zu einem lokalen Minimum, wird dieser zwangsweise auf die TabuListe gesetzt und anschließend kann es dazu kommen, dass kein besserer Wert gefunden werden kann. Da der letzte übergebene Wert jedoch auf der TabuListe steht, muss der Algorithmus sich verschlechtern und entkommt sofern die TabuListe lang genug ist, dem lokalen Optimum. Dies ist praktisch auch sehr gut an den größeren logistischen Problemen zu erkennen. Bei der Datei 'br17.json' jedoch nicht, da hier zu viele Permutationen die selben sehr guten Ergebnisse liefern. Der Sinn dieses Vergehens ist simpel, dem lokalen Optimum zu entkommen.

Abschließend wird das langfristige Gedächtnis aka solutPoool mit der selben einfachen Logik mit 'zwischenspeicher' komplett durchsucht und das globale Optium, das in dieser Rechenzeit mit der MetaHeuristik erreicht werden konnte, zurückgegeben. Zurückgegeben einerseits in Form von einem Output in der Konsole, aber auch gleichzeitig in Form der letzten Klasse: in der Datei OutputData.py mit der Klasse outputAlgorithm und der Funktion startOutput.

In [ ]:
        #Abbruchkriterium wurde erreicht, Zeit ist abgelaufen: durchsuche gesamten LangzeitGedächtnis - Lösungsraum
        zwischenSpeicher = []
        for i in self.solutPoool:
            zwischenSpeicher.append(EvaluationLogic().SolutionFinder(i, self.matrix)) #errechne für alle in SolutionPool enthaltenen Lösungen die Weglänge

        index_min = min(range(len(zwischenSpeicher)), key=zwischenSpeicher.__getitem__) #Suche nach dem Index mit dem geringsten Lösungswert
        bestFinalSolution = deepcopy(self.solutPoool[index_min]) #die gesamte-beste Lösung

        print(f'\nDie beste globale Lösung der MetaHeuristik ist {bestFinalSolution} mit {EvaluationLogic().SolutionFinder(deepcopy(bestFinalSolution), self.matrix)}.')
        print(f'Dafür hat die MetaHeuristik {timer} Sekunden an Rechenzeit benötigt.')

        outputAlgorithm(bestFinalSolution, self.matrix, self.path).startOutput() #starte Output auf Basis der gesamt-besten Lösung
        
        return bestFinalSolution
\\ #diese zwei Zeichen bitte nicht beachten

---

Die Klasse outputAlgorithm wird mit den Werten der finalen besten Permutation, der Wegematrix und dem Namen der Datei instanziiert. Der Algorithmus prüft zuerst, ob das Verzeichnis \\Solutions bereitsexistiert und falls nicht, wird dieses noch generiert. Bitte beachten Sie die Kommentare, die den Code erklären. Um eine einfacherere Ansicht zu erhalten, gehen Sie bitte in die Datei OutputData.py.

In [ ]:
class
    def startOutput(self):

        if not os.path.exists('Solutions'):
            os.mkdir('Solutions')

        #Einführen von Zählvariablen zur Vorbereitung des Outputs der Lösung
        zaehlvar = 1
        writer_path = 'Solutions\Solution-' + self.path.replace('.json', '') + '.csv' #Generieren des Output-Paths auf Basis der Input-Datei, dabei muss die Dateiendung der Input-Datei gelöscht werden
        cum_sum = 0 #Einführen der kumulierten Summe zum Abspeichern der CSV-Datei

        #Generierung einer neuen Datei oder Überschreibung einer bereits existierenden Lösung mit Feldnamen 'Id' und 'Distances'
        with open(writer_path, 'w', newline='') as file: #'w' damit vorhandene Datei überschrieben/resettet wird, newline = '' damit keine Absätze nach jeder Zeile generiert werden
            dw = csv.DictWriter(file, delimiter=';', fieldnames=['Id', 'Distance']) #Trennungszeichen ';' eingeführt und Feldnamen eingegeben
            dw.writeheader()

        #Output/CSV-Generierung mithilfe der besten Lösung/Permutation aus Solver-Algorithmus
        for i in self.order_final: #Für jedes Element der finalen Lösung wird der Weg nochmals nachvollzogen und in CSV-Datei exportiert. Beschreibung der Schleife siehe Solver
            if zaehlvar == len(self.matrix[0]):
                cum_sum = cum_sum + self.matrix.iloc[i, self.order_final[(zaehlvar)-1]]   #Beim ersten Abbruchkriterium aufgrund des letzten Elementes des Arrays muss (order_final[(zaehlvar)-1) gecoded werden, 
                                                                                #da als letzte Instanz in der vorherigen Iteration die (zaehlvar) um eins erhöht wurde und dies rückgängig gemacht werden muss

                with open(writer_path, mode='a', newline='') as file: #mode='a' damit die bereits existierende CSV-Datei nur erweitert (appended) wird. Ein 'w' würde alle bisherigen Einträge löschen
                    write_row = csv.writer(file, delimiter=';')
                    write_row.writerow([i, cum_sum])
                break
            elif self.matrix.iloc[i, self.order_final[zaehlvar]] == -1:
                with open(writer_path, mode='a', newline='') as file: #mode='a' damit die bereits existierende CSV-Datei nur erweitert (appended) wird. Ein 'w' würde alle bisherigen Einträge löschen
                    write_row = csv.writer(file, delimiter=';')
                    write_row.writerow(['Path is not valid']) #Abbruchkriterium falls 'ideale Lösung' des Solvers nicht erlaubt ist 
                zaehlvar +=1
                break    
            else:
                with open(writer_path, mode='a', newline='') as file: #mode='a' damit die bereits existierende CSV-Datei nur erweitert (appended) wird. Ein 'w' würde alle bisherigen Einträge löschen
                    write_row = csv.writer(file, delimiter=';')
                    write_row.writerow([i, cum_sum])

                cum_sum = cum_sum + self.matrix.iloc[i, self.order_final[zaehlvar]]
                zaehlvar += 1 
\\ #diese zwei Zeichen bitte nicht beachten

Die Output Datei, die erzeugt wird ist entsprechend den Vorgaben eine .csv Textdatei, die mit Semikolon separiert einerseits die KnotenID und danach die kumulierte Distanz angibt. Als Beispiel ein Output für br17.json:

In [ ]:
Id;Distance
0;0
6;8
3;14
4;14
15;20
5;20
14;20
9;28
13;31
2;31
12;34
1;34
10;34
8;39
16;39
7;39
11;44
17;44

---

Die besten Werte, die während der gesamten Erprobungszeit erreicht wurden sind folgende:

In [ ]:
br17.json:
39      mit     [0, 16, 8, 7, 4, 3, 14, 6, 15, 5, 1, 12, 9, 10, 13, 2, 11, 17]

ESC25.json:
1836    mit     [0, 1, 12, 25, 3, 9, 22, 23, 24, 21, 17, 11, 16, 2, 13, 20, 8, 10, 19, 5, 7, 14, 6, 4, 18, 15, 26]

ft70.json:
40867   mit     [0, 1, 60, 14, 63, 66, 64, 69, 65, 68, 67, 48, 43, 49, 35, 38, 39, 37, 36, 42, 41, 40, 62, 58, 57, 44, 46, 45, 61, 59, 10, 9, 8, 7, 13, 12, 11, 5, 4, 22, 25, 3, 30, 15, 27, 24, 2, 28, 54, 52, 55, 51, 50, 18, 31, 34, 23, 56, 53, 20, 17, 16, 19, 47, 33, 32, 29, 21, 26, 6, 70]

rbg109a.json:
1618    mit     [0, 1, 3, 2, 8, 5, 6, 7, 4, 9, 11, 10, 22, 12, 13, 14, 15, 16, 21, 17, 18, 24, 19, 20, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 37, 34, 35, 39, 36, 38, 40, 41, 42, 43, 44, 59, 45, 46, 47, 49, 54, 48, 50, 51, 52, 53, 55, 56, 57, 58, 60, 61, 69, 62, 63, 64, 65, 66, 67, 68, 70, 71, 72, 73, 74, 75, 76, 80, 77, 78, 79, 81, 82, 83, 84, 85, 86, 87, 88, 91, 89, 92, 90, 93, 94, 95, 97, 96, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110]

ry48p.json:
15691   mit     [0, 8, 37, 30, 43, 17, 35, 6, 5, 18, 26, 16, 42, 29, 36, 27, 45, 14, 11, 32, 19, 46, 20, 12, 22, 10, 39, 21, 2, 13, 24, 4, 47, 38, 31, 23, 9, 34, 44, 25, 3, 1, 41, 28, 33, 40, 15, 7, 48]

Die besten Werte, die mit den aktuellen Parametern erreicht werden sind:

In [ ]:
br17.json:      44
ESC25.json:     2318
ft70.json:      42004
rbg109a.json:   2075
ry48p.json:     17180

---

Auf Basis der vielen Probedurchläufe und im Vergleich mit Kommilitonen und anderen Algorithmen würde ich das Potential des TabuSearch als sehr groß einschätzen. Die aktuelle Lösungsgüte mit den gewählten Parametern ist jedoch lediglich gut bis befriedigend. Für kleinere Instanzen gibt der Algorithmus schnell gute Lösungen zurück, jedoch meist nur, wenn die Startlösung bereits gut gewesen ist. Das Überwinden der lokalen Optima ist eine große Stärke, die TabuSearch mit den gewählten Paramtern und der Art der Implementierung jedoch nicht wirklich ausnutzen kann. Die Intensivierung des Algorithmus ist sehr Rechenaufwändig und die Programmierweise nicht ressourcensparend genug.<br>
<br>
Verwendet wurde für die Programmierung und Testung des Algorithmus ein Microsoft Surface Pro der 6. Generation mit 8GB RAM und einem Intel Core i5-8250U mit 1,6 bzw. 1,8 GHz. Dies entspricht einem Mittelklasse Office Rechner.

Parameter Erläuterung:
es gibt zwei wesentliche Paramter, die variiert werden können. Dabei handelt es sich um 1) maxTabuListLength und um 2) maxIterations.<br>
1) Die Länge der TabuListe wird in der Literatur nicht klar bestimmt. Da Die Länge der TabuListe Cycling verhindern soll, ist sie verhältnismäßig wichtig. Es gibt verschiedene Konzepte, unteranderem wird in Glover et al. festgestellt, dass eine TabuListe proportional zur Problem Größe implementiert werden soll [2]. Außerdem wurd in [2] festgestellt, dass sich eine dynamisch ändernde TabuListen-Länge nur positiv auf das Ergebnis auswirkt. In dieser Arbeit wurde jedoch aufgrund der Reproduzierbarkeit der Lösungen darauf verzichtet und eine statische Länge implementiert: 35. Dieser Wert enstspringt experimentellen Testens, wie [3] es nahelegt. Ein zu geringer Wert riskiert, dass nach einer geringen Anzahl an Tauschen die selbe lokale beste Lösung wieder gefunden wird und die Diversifikation dadurch kaum zum Einsatz kommt. Ein zu hoher Wert könnte die Intensivierung blockieren, bessere Lösung in einer nahegelegenen Nachbarschaft zu finden. <br>
2) Die Intensivierung der Lösungsfindung wird maßgeblich durch die Nachbarschaftssuche beeinflusst. Diese Intensivierungsphase der Lösungsfindung ist entscheidend und wird durch die maxIterations gesteuert. Diese gibt an, nach wievielen Tauschen in der Nachbarschaft die Intensivierung der Lösung abgebrochen werden soll. Ist der Wert zu hoch, benötigt der Rechner enorm lange, besonders bei größeren Mengen an Knoten (50+), und bei zu niedrigen maxIterations-Werten kann selten eine wirklich gute lokale beste Lösung gefunden werden. Es muss eine Balance dazwischen gefunden werden: 500 Iterationen liefern relativ gute Ergebnisse. Dieser Wert wurde erneut experimentell bestimmt. Als Beispiel wäre die Wahl von 1000 Iterationen bei der rbg109a.json Datei sehr unvorteilhaft, da 1000 Permutationen generiert und anschließend auf ihre Güte getestet werden. Bei dieser Menge als Rechnerleistung ist dies sehr langsam und führt dazu, dass in der vorgegebenen Zeit deutlich weniger "neue Ansätz" bzw. Iterationen mit der letzt-besten Permutation getätigt werden können. Es kommt am Ende ein schlechteres Ergebnis zurück als bei 500 Iterationen, die Intensivierung des Algorithmus ist zu hoch im Vergleich zur Diversifikation.

<br>
Bestimmung der Parametereinstellung nach folgender Form:<br>
Funktionswert: Anzahl Itertationen (Parametereinstellung)<br>

MaxIterations Parametereinstellungen:<br>
<br>
br17.json<br>
44: 34 Iterationen (500)<br>
44: 33 Iterationen (1000)<br>
<br>
ESC25.json<br>
2318: 9 Iterationen (500)<br>
2318: 9 Iterationen (1000)<br>
<br>
ry48p.json<br>
17474: 3 Iterationen (500)<br>
17775: 2 Iterationen (1000)<br>
<br>
ft70<br>
42126: 15 Iterationen (500)<br>
42400:  8 Iterationen (1000)<br>
<br>
rbg109a.json<br>
2075: 69 Iterationen (500)<br>
1968: 16 Iterationen (1000)<br>

<br>

MaxTabuListLength:<br>
<br>
br17.json<br>
44: 33 Iterationen (2)<br>
44: 33 Iterationen (5)<br>
44: 33 Iterationen (35)<br>
44: 33 Iterationen (50)<br>
<br>
ESC25.json<br>
2318: 8 Iterationen (2)<br>
2318: 8 Iterationen (5)<br>
2318: 8 Iterationen (35)<br>
2318: 9 Iterationen (50)<br>
<br>
ry48p.json<br>
17474: 3 Iterationen (2)<br>
17474: 3 Iterationen (5)<br>
17180: 4 Iterationen (35)<br>
17474: 3 Iterationen (50)<br>
<br>
ft70.json<br>
42126: 18 Iterationen (2)<br>
42126: 16 Iterationen (5)<br>
42004: 18 Iterationen (35)<br>
42004: 19 Iterationen (50)<br>
<br>
rbg109a.json<br>
2075: 34 Iterationen (2)<br>
2075: 38 Iterationen (5)<br>
2075: 37 Iterationen (35)<br>
2075: 37 Iterationen (50)<br>

<br>
<br>

Nach dieser Parameteruntersuchung wurde sich für die folgenden Parameter entschieden:<br>
maxIterations: 500<br>
maxTabuSearchListLength: 35<br>
<br>
Erweiterungsmöglichkeiten des ALgorithmus wurden bereits angesprochen. Zum Einen gibt es die Möglichkeit, dynamische Parameter zu nutzen und die TabuListLength auf Basis der Häufigkeit der Erfassung der selben Lösung dahingegend anzupassen, dass sich die Länge immer weiter erhöht, wenn keine Verbesserungen gefunden werden können.<br>
Zudem lässt sich das bereits eingebaute Langzeitgedächtnis mit den besten lokalen Lösungen nutzen, um von den besten bisherigen Lösungen die Suche neuzustarten, mit veränderten Parametern etc.<br>
In der Literatur wird ebenfalls von einem mittelfristigen Gedächtnis gesprochen, mit dem gute Ergenisse noch verbessert werden könnten [4].

Literaturverzeichnis:<br>
<br>
[1]: S. Basu, "Tabu Search Implementation on Traveling Salesman Problem and Its Variations: A Litertaure Survey", Amercian Journal of Operations Research, 2012, pp. 163-173.<br>
[2]: F. Glover, E. Taillard, D. de Werra, "A user's guide to tabu search", Ann Oper Res 41, 1993, pp. 1-28.<br>
[3]: M. Dam, M. Zachariasen, "Tabu Search on the Geometric Traveling Saleman Problem", Master of Science Thesis, 1994.<br>
[4]: M. Gendreau, J.-Y. Potvin, "Handbook of Metaheuristics, Third Edition", Springer Cham, 2019.<br>
<br>
weitere Literatur, die zum Verständnis und Aufbau hinzugezogen wurde:<br>
<br>
[A]: A.Banerjee, D. Singh, S. Sahana, I. Nath, "Cognitive Big Data Intelligence with a Metaheuristic Approach - Chapter 3 - Impacts of metaheuristic and swarm intelligence approach in optimization", Academic Press, 2022, pp. 71-99.<br>
[B]: F. Mascia, P. Pellegrini, T. Stützle, M. Birattari, "An Analysis of Parameter Adaption in Reactive TabuSearch", IRIDIA - Technical Report Series, Technical Report No. TR/IRIDIA/2011-026, 2016.<br>
[C]: S. Stepanenko, "Global Optimization Methods based on Tabu Search", Dissertation at Julius-Maximilians-Universität Würzburg, 2008.<br>
[D]: W. Jaziri, "Local Search Techniques: Focus on Tabu Search", IN TECH d.o.o., 2008.<br>
[E]: S. Sarmady, "An Investigation on Tabu Search Parameters", Universiti Sains Malaysia, 2007.<br>
[F]: F. Geyik, I. H. Cedimoglu, "The strategies and parameters of tabu search for job-shop scheduling", Journal of Intelligent Manufacturing, 15, 2004, pp. 439-448.<br>



-----------------------

In nachfolgender Zelle ist der vollständige Algorithmus durch importe durchführbar:

In [1]:
import time
import threading
from InputData import *
from EvaluationLogic import *
from OutputData import *
from Solver import *
from BestStartSolution import *


def process_scenario(start, end):
    for i in range(start,end):
        start_time = time.time()  # Start timing before the process begins

        path = 'scenario_example_id_' + str(i) + '.json'
        seedNumber = 50
        maxTabuListLength = 35
        maxIterations = 500

        data = InputData(path)
        firstSolution = BestStartSolution(data.matrix, seedNumber).generate_best_start_solution()

        solution = Solver(path).tabuSearch(firstSolution, data.matrix, path, maxTabuListLength, maxIterations)

        end_time = time.time()  # End timing after the process ends
        duration = end_time - start_time  # Calculate the duration

        print(f"Iteration {i}: Duration {duration:.2f} seconds")  # Print the duration for each iteration

threads = []
for i in range(1, 16):
    t = threading.Thread(target=process_scenario, args=(i*16,i*16+16,))
    t.start()
    threads.append(t)

for t in threads:
    t.join()


Iteration 160: Duration 17.60 seconds
Iteration 16: Duration 18.04 seconds
Iteration 240: Duration 18.76 seconds
Iteration 48: Duration 19.47 seconds
Iteration 64: Duration 19.99 seconds
Iteration 192: Duration 20.10 seconds
Iteration 176: Duration 20.43 seconds
Iteration 208: Duration 20.34 seconds
Iteration 32: Duration 21.08 seconds
Iteration 96: Duration 21.03 seconds
Iteration 80: Duration 21.29 seconds
Iteration 112: Duration 21.54 seconds
Iteration 224: Duration 21.27 seconds
Iteration 144: Duration 22.35 seconds
Iteration 128: Duration 34.83 seconds
Iteration 193: Duration 18.31 seconds
Iteration 161: Duration 21.85 seconds
Iteration 241: Duration 20.92 seconds
Iteration 81: Duration 19.66 seconds
Iteration 49: Duration 22.28 seconds
Iteration 65: Duration 21.96 seconds
Iteration 97: Duration 22.62 seconds
Iteration 113: Duration 22.53 seconds
Iteration 209: Duration 24.34 seconds
Iteration 177: Duration 26.55 seconds
Iteration 17: Duration 33.19 seconds
Iteration 33: Duration 